In [1]:
!mkdir -p vncorenlp/models/wordsegmenter

!curl -O https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!curl -O https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!curl -O https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr

# Di chuyển các file vào đúng thư mục
!mv VnCoreNLP-1.1.1.jar vncorenlp/
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 26.1M  100 26.1M    0     0  36.6M      0 --:--:-- --:--:-- --:--:-- 36.6M
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  514k  100  514k    0     0  1572k      0 --:--:-- --:--:-- --:--:-- 1577k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  125k  100  125k    0     0   513k      0 --:--:-- --:--:-- --:--:--  514k


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install vncorenlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 24.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645932 sha256=00d59d6f63b2967a7d4150ec8af113c2e86c4970db19e9af4ee16cd81ab591d4
  Stored in directory: /root/.cache/pip/wheels/5d/d9/b3/41f6c6b1ab758561fd4aab55dc0480b9d7a131c6aaa573a3fa
Successfully built vncorenlp


In [16]:
from vncorenlp import VnCoreNLP

vncorenlp = VnCoreNLP("vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')

In [17]:
import pandas as pd

# Đọc file output.csv để tạo từ điển thay thế
file_path = "/content/cleaned_output.csv"  # Thay đường dẫn file nếu cần
replacement_df = pd.read_csv(file_path)
replacement_df

,Column 1,Column 2
0,dấu bé 3,<3
1,dấu bé ba,<3
2,ă,á
3,â,á
4,ák,á
...,...,...
2619,đ,đéo
2620,chưi,chửi
2621,Vaicalon,Vãi cả lồn
2622,vaicalon,vãi cả lồn


In [38]:
# Đảm bảo sử dụng đúng tên cột trong file
abbreviation_column = "Column 1"  # Thay bằng tên cột chứa từ viết tắt
replacement_column = "Column 2"  # Thay bằng tên cột chứa từ thay thế

# Tạo từ điển từ file CSV
replacement_dict = dict(zip(replacement_df[abbreviation_column], replacement_df[replacement_column]))

# Hàm thay thế các từ viết tắt
def replace_abbreviations(train_sentence, replacement_dict):
    if not isinstance(train_sentence, str):  # Kiểm tra nếu dữ liệu không phải là chuỗi
        return train_sentence
    words = train_sentence.split()  # Tách câu thành các từ
    replaced_words = [replacement_dict.get(word, word) for word in words]  # Thay thế từ
    return ' '.join(replaced_words)  # Ghép câu lại

# Đọc tệp CSV của dữ liệu gốc
train = "/content/Train_aug.csv"
# dev = "/content/dev.xlsx"
# test = "/content/test.xlsx"

train_df = pd.read_csv(train)
# dev_df = pd.read_csv(dev)
# test_df = pd.read_csv(test)

# Kiểm tra và xử lý các giá trị bị thiếu trong cột 'free_text'
train_df['content'] = train_df['content'].fillna("")  # Thay giá trị NaN bằng chuỗi rỗng
# dev_df['content'] = dev_df['content'].fillna("")
# test_df['content'] = test_df['content'].fillna("")

# Áp dụng hàm thay thế để chuẩn hóa dữ liệu từ cột 'free_text'
train_df['content'] = train_df['content'].apply(lambda x: replace_abbreviations(x, replacement_dict))
# dev_df['content'] = dev_df['content'].apply(lambda x: replace_abbreviations(x, replacement_dict))
# test_df['content'] = test_df['content'].apply(lambda x: replace_abbreviations(x, replacement_dict))



In [39]:
import re
import numpy as np

STOPWORDS = '/content/vietnamese-stopwords.txt'
with open(STOPWORDS, "r") as ins:
    stopwords = []
    for line in ins:
        dd = line.strip('\n')
        stopwords.append(dd)
    stopwords = set(stopwords)

def filter_stop_words(train_sentences, stop_words):
    new_sent = [word for word in train_sentences.split() if word not in stop_words]
    train_sentences = ' '.join(new_sent)

    return train_sentences

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

def preprocess(text, tokenized=True, lowercased=True):
    # text = ViTokenizer.tokenize(text)
    # text = ' '.join(vncorenlp.tokenize(text)[0])
    text = filter_stop_words(text, stopwords)
    text = deEmojify(text)
    text = text.lower() if lowercased else text
    if tokenized:
        pre_text = ""
        sentences = vncorenlp.tokenize(text)
        for sentence in sentences:
            sentence_replace = [text.replace("_", " ")for text in sentence]
            pre_text += " ".join(sentence_replace)
        text = pre_text
    return text

def pre_process_features(X, y, tokenized=True, lowercased=True):
    X = [preprocess(str(p), tokenized=tokenized, lowercased=lowercased) for p in list(X)]
    for idx, ele in enumerate(X):
        if not ele:
            np.delete(X, idx)
            np.delete(y, idx)
    X_df = pd.DataFrame(X, columns = ['content'])
    return pd.concat([X_df, y], axis = 1)

In [40]:
train_data = pre_process_features(train_df['content'], train_df[['individual', 'groups', 'religion/creed', 'race/ethnicity', 'politics']])
# dev_data = pre_process_features(dev_df['content'], dev_df[['individual', 'groups', 'religion/creed', 'race/ethnicity', 'politics']])
# test_data = pre_process_features(test_df['content'], test_df[['individual', 'groups', 'religion/creed', 'race/ethnicity', 'politics']])

In [41]:
train_data.to_csv("Train_aug_normalize.csv")
# dev_data.to_csv("Dev_normalize.csv")
# test_data.to_csv("Test_normalize.csv")